# Imports

In [ ]:
import pandas as pd
import numpy as np


# Load Data

In [ ]:
# Training Data


# Features


# Baseline

## Today for tomorrow

## Moving Average

# Model Comparison

## Traditional

### ARIMA/SARIMA

### Exponential Smoothing

## Machine Learning